In [226]:
import keras as k
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering

In [227]:
#read in service data
service_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/service_cost.csv')
service_data.tail()

parts_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/cleaned_parts_data.csv')

In [228]:
index_to_drop_ = 2381
service_data.drop(index_to_drop_, axis=0, inplace=True)
index_to_drop = 2382
service_data.drop(index_to_drop, axis=0, inplace=True)


In [229]:
def clean_and_convert_columns(df, columns, target_dtype):
    """
    Clean and convert specified columns in a DataFrame to the target data type.

    Args:
        df (pd.DataFrame): DataFrame to clean and convert columns.
        columns (list of str): List of column names to clean and convert.
        target_dtype (type): Target data type to convert columns to.

    Returns:
        pd.DataFrame: Cleaned DataFrame with specified columns converted.
    """
    for column in columns:
        df[column] = df[column].str.replace(',', '')
        df[column] = df[column].astype(target_dtype)
    return df

# Columns to clean and convert
columns_to_clean = service_data.columns  # Replace with the actual column names

# Clean and convert columns to float
service_data = clean_and_convert_columns(service_data, columns_to_clean, float)

In [230]:
service_data.dtypes

Parts Cost            float64
Labor Cost            float64
Sublet Cost           float64
Total Service Cost    float64
dtype: object

In [231]:
avg_service_cost = service_data['Total Service Cost'].mean()
print(f'Average Service Cost: ${round(avg_service_cost, 2)}')

Average Service Cost: $322.86


In [232]:
time_period = ['Sales - Jan', 'Sales - Feb',
       'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
       'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct',
       'Sales - Nov', 'Sales - Dec', 'Sales - 1st Qtr', 'Sales - 2nd Qtr',
       'Sales - 3rd Qtr', 'Sales - 4th Qtr', 'Sales - Last Year']

In [233]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def find_best_alpha(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    alpha_values = np.linspace(0.01, 0.99, num=20)

    best_alpha = None
    best_mse = float('inf')

    for alpha in alpha_values:
        model = ExponentialSmoothing(y_train, seasonal='add', seasonal_periods=12, trend='add')
        fit_model = model.fit(smoothing_level=alpha)
        forecast = fit_model.forecast(steps=len(X_test))

        mse = mean_squared_error(y_test, forecast)
        if mse < best_mse:
            best_mse = mse
            best_alpha = alpha

    return best_alpha, best_mse

# Replace 'time_period' with the actual column name for your X data
time_period = 'Sales - Last Year'  # Change this to your desired column
X = parts_data[time_period]
y = parts_data['Sales - This Year']

best_alpha, best_mse = find_best_alpha(X, y)
print(f"Best Alpha: {best_alpha}, Best MSE: {best_mse}")


/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(da

Best Alpha: 0.2163157894736842, Best MSE: 48.557361736163436


EOQ = sqrt(2*demand*ordering cost / carrying cost)

demand = use exponential smoothing to forecast demand Ft = a*Xt = (1-a)*Ft-1

ordering cost = frequency * (cost of the part + cost of transportation + lead time costs)

carrying cost = storage cost + capital cost + obsolecence + depreciation + opportunity cost

In [242]:
def calculate_demand(sales_data, alpha):
    demand = sales_data
    initial_forecast = sales_data * 1.15
    forecast = alpha * demand + (1 - alpha) * initial_forecast

    return forecast

period = 'Sales - This Year'
alpha = 0.2163157894736842
parts_data['Demand'] = calculate_demand(parts_data[period], alpha)


In [243]:
def calculate_ordering_costs(order_quantity, fixed_ordering_costs, carrying_cost_rate, avg_lead_time_days, sales_data):
    lead_time_demand = (sales_data / len(sales_data)) * avg_lead_time_days
    carrying_costs_lead_time = 0.5 * carrying_cost_rate * lead_time_demand  

    total_ordering_costs = fixed_ordering_costs + carrying_costs_lead_time * order_quantity
    
    return total_ordering_costs

order_quantity = parts_data['Quantity Ordered YTD'].sum()
fixed_ordering_cost = parts_data['Total Cost']* 0.3
carrying_cost_rate = 0.20  
avg_lead_time_days = 14  
sales_data = parts_data['Sales - This Year'] 

# Calculate total ordering costs considering lead time
parts_data['Ordering Cost'] = calculate_ordering_costs(order_quantity, fixed_ordering_cost, carrying_cost_rate, avg_lead_time_days, sales_data)



In [244]:
def calculate_carrying_cost(average_inventory_level, carrying_cost_rate, avg_time_in_inventory, depreciation):
    """
    Calculate carrying cost for an inventory item.
    
    Parameters:
    average_inventory_level (float): Average inventory level in units.
    carrying_cost_rate (float): Carrying cost rate as a decimal.
    
    Returns:
    float: Carrying cost for the inventory item.
    """
    carrying_cost = average_inventory_level * carrying_cost_rate * avg_time_in_inventory + average_inventory_level * depreciation

    return carrying_cost

# Example usage
average_inventory_level = parts_data['Average Inventory']
carrying_cost_rate = 0.125
depreciation = 0.1
avg_time_in_inventory = parts_data['Months No Sale']

parts_data['Carrying Cost'] = calculate_carrying_cost(average_inventory_level, carrying_cost_rate, avg_time_in_inventory, depreciation)


In [245]:
import math

def EOQ(dataframe, demand_col, ordering_cost_col, carrying_cost_col):
    dataframe['EOQ'] = dataframe.apply(
        lambda row: math.sqrt((2 * row[demand_col] * row[ordering_cost_col]) / row[carrying_cost_col])
                     if row[carrying_cost_col] > 0 else row[demand_col] * math.sqrt(2 * row[ordering_cost_col]),
        axis=1
    )
    return dataframe

demand_col = 'Demand'
ordering_cost_col = 'Ordering Cost'
carrying_cost_col = 'Carrying Cost'

parts_data = EOQ(parts_data, demand_col, ordering_cost_col, carrying_cost_col)

parts_data.to_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/parts_data_functions.csv', index=False)

In [247]:
mask = parts_data['EOQ'] > 0

cols = ['Part Number', 'Category', 'Description', 'Supplier Name', 'Margin %',
       'Margin', 'Price', 'Quantity', 'Sales Last Month',
       'Sales Last 3 Months', 'Sales Last 6 Months', 'Sales Last 9 Months',
       'Sales Last 12 Months', 'Sales - 1st Qtr', 'Sales - 2nd Qtr',
       'Sales - 3rd Qtr', 'Sales - 4th Qtr', 'Sales - This Year',
       'Sales - Last Year','COGS','Average Inventory',
       'Turnover', 'Demand', 'Ordering Cost', 'Carrying Cost', 'EOQ']
parts_data[mask][cols]

,Part Number,Category,Description,Supplier Name,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,...,Sales - 4th Qtr,Sales - This Year,Sales - Last Year,COGS,Average Inventory,Turnover,Demand,Ordering Cost,Carrying Cost,EOQ
17,002-10703,ACC,exhaust spring 111mm ea,moto international,100.00000,1.99,1.99,0,0,0,...,0,20,0,6.0000,3.0000,2.000000,22.351053,1.487245,2.550000,5.106051
30,004HF137,ACC,hi-flo o-filt suz 16510-37450,thibault canada,100.00000,6.99,6.99,0,0,0,...,0,1,0,4.0900,2.0450,2.000000,1.117553,0.074362,0.971375,0.413649
63,0069922BC,ACC,new style universal cruise ctr,thibault canada,50.02633,9.50,18.99,2,0,0,...,0,1,1,9.4900,42.7250,0.222118,1.117553,5.768362,30.975625,0.645157
64,0069925B,ACC,"universal cruise control 1""",thibault canada,50.02633,9.50,18.99,1,1,1,...,0,1,0,9.4900,23.7350,0.399831,1.117553,2.921362,2.373500,1.658621
65,007-18704,ACC,main jet cariers & wrench,moto international,100.00000,13.99,13.99,0,0,0,...,0,3,5,4.2000,2.1000,2.000000,3.352658,0.223087,1.785000,0.915435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89145,YTX7L-BS,ACC,yuasa ytx7l-bs [ see also-34-510,moto international,100.00000,69.99,69.99,0,0,0,...,0,2,1,90.9400,45.4700,2.000000,2.235105,0.148725,4.547000,0.382378
89147,YTX9-BS,ABT,battery,interstate battery,100.00000,110.99,110.99,0,0,0,...,0,1,2,77.5600,38.7800,2.000000,1.117553,0.074362,987.920500,0.012971
89148,YTZ10S,ABT,yuasa ytz10s,moto international,100.00000,174.99,174.99,0,0,0,...,0,1,3,109.0200,54.5100,2.000000,1.117553,0.074362,5.451000,0.174617
89149,YTZ12S,ABT,ytz12s factory activated,parts canada/power twins,100.00000,299.99,299.99,0,0,0,...,0,1,0,209.9600,104.9800,2.000000,1.117553,0.074362,272.948000,0.024677


In [248]:
def custom_loss_function(y_true, y_pred):
    #define the function here:
    # customer LTV: avg_service_cost + avg(add ons cost) + avg(cost of tires) + avg(cost to fix drops)* % likelihood of w_turnover * normalized_turnover + w_eoq * normalized_eoq
    #Obsolecence cost: 95% * (cost per unit)
    
    squared_difference = tf.square(y_true - y_pred)
    mean_squared_error = tf.reduce_mean(squared_difference)

    return mean_squared_error

In [249]:
def custom_distance_function(x, y):
    #create custom distance function here
    return None